<a href="https://colab.research.google.com/github/sutharimanikanta/Golab/blob/main/Untitled12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/SartajBhuvaji/Brain-Tumor-Classification-DataSet

Cloning into 'Brain-Tumor-Classification-DataSet'...
remote: Enumerating objects: 3039, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 3039 (delta 0), reused 0 (delta 0), pack-reused 3035
Receiving objects: 100% (3039/3039), 79.25 MiB | 27.69 MiB/s, done.
Updating files: 100% (3264/3264), done.


In [ ]:



import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import pickle
import time
import numpy as np
import keras.optimizers
from sklearn.metrics import classification_report
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import imutils
import cv2
import PIL.Image
import os
%matplotlib inline

demo_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    rescale=1./255,
    shear_range=0.05,
    brightness_range=[0.1, 1.5],
    horizontal_flip=True,
    vertical_flip=True
)
import os

# Define categories
categories = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]

# Define paths for training and testing data
train_dir = "./data/train"
test_dir = "./data/test"

# Function to create directories
def make_directories(base_dir):
    for category in categories:
        os.makedirs(os.path.join(base_dir, category), exist_ok=True)

# Create directories for training data
make_directories(train_dir)

# Create directory for testing data
os.makedirs(test_dir, exist_ok=True)
import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from sklearn.model_selection import train_test_split

def cropAndAugmentation():
    # Augmentation Code
    IMG_SIZE = 80
    ADD_PIXELS = 5  # Adjust as needed
    dim = (IMG_SIZE, IMG_SIZE)
    cwd = os.getcwd()
    directory = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]
    desired_folder = "Brain-Tumor-Classification-DataSet/Training/"

    # Create copies of the original dataset with different modifications
    datasets = []
    for i in range(3):
        datasets.append([])

    for input_folder in directory:
        input_folder_path = os.path.join(cwd, desired_folder + input_folder)
        if not os.path.exists(input_folder_path):
            raise FileNotFoundError(f"Input folder {input_folder_path} does not exist.")

        for img in os.listdir(input_folder_path):
            image_path = os.path.join(input_folder_path, img)
            image = cv2.imread(image_path)

            # Resize images for the three datasets
            resized_image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
            rotated_image = cv2.rotate(resized_image, cv2.ROTATE_90_CLOCKWISE)
            flipped_image = cv2.flip(resized_image, 1)  # Flip along the vertical direction

            # Append resized images to datasets
            datasets[0].append(resized_image)
            datasets[1].append(rotated_image)
            datasets[2].append(flipped_image)

    # Concatenate the three datasets into one
    concatenated_dataset = np.concatenate(datasets, axis=0)

    # Split the dataset into training and testing sets
    X_train, X_test = train_test_split(concatenated_dataset, test_size=0.1, random_state=42)
    # Create an iterator for training data with augmentation
    train_iterator = demo_datagen.flow(X_train, batch_size=32, shuffle=True)

    # Store training and testing data
    data_dir = "./data"
    train_dir = os.path.join(data_dir, "train")
    test_dir = os.path.join(data_dir, "test")

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Store training images
    for idx, img_batch in enumerate(train_iterator):
        for img in img_batch:
            category_folder = directory[idx % len(directory)]
            img_path = os.path.join(train_dir, category_folder, f"image_{idx}.jpg")
            os.makedirs(os.path.join(train_dir, category_folder), exist_ok=True)
            cv2.imwrite(img_path, img)
            idx += 1  # Increment index for the next image

        if idx >= len(X_train):  # Stop when all original training images are processed
            break

    # Store testing images
    for idx, img in enumerate(X_test):
        img_path = os.path.join(test_dir, f"image_{idx}.jpg")
        cv2.imwrite(img_path, img)
    # Count total number of images
    total_images = len(X_train) + len(X_test)
    print("Total number of images:", total_images)
    print("Number of training images:", len(X_train))
    print("Number of testing images:", len(X_test))

# Execute the function
cropAndAugmentation()
TEST_DIR = '/content/data/test' # test data folder
TRAIN_DIR = '/content/data/train' # train data folder
CATEGORIES =["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]
from tqdm import tqdm
import random
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(TRAIN_DIR, category)  # create path
        class_num = CATEGORIES.index(category)  # get the classification
        for img in tqdm(os.listdir(path)):
            # iterate over each image per category
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)  # convert to array
            training_data.append([img_array, class_num])  # add this to our training_data

    random.shuffle(training_data)

create_training_data()
np.save('train_data.npy', training_data)
print(len(training_data))
print("train data")
print()
testing_data=[]

def create_testing_data():
    for img in tqdm(os.listdir(TEST_DIR)):
        img_path = os.path.join(TEST_DIR, img)
        if os.path.isfile(img_path):
            img_array = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img_array is not None:
                class_num = 0  # You may need to adjust this if you have multiple classes
                testing_data.append([img_array, class_num])
            else:
                print(f"Failed to read image: {img_path}")
        else:
            print(f"Not a file: {img_path}")

    random.shuffle(testing_data)
    print("Total number of images in testing data:", len(testing_data))

create_testing_data()
np.save('test_data.npy',testing_data)
print(len(testing_data))
print("test data")
print()
 #Convert to NumPy array
X_train = np.array([i[0] for i in training_data])
Y_train = np.array([i[1] for i in training_data])
# Convert to NumPy array
X_test = np.array([i[0] for i in testing_data])
Y_test = np.array([i[1] for i in testing_data])
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Activation, Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import time
import numpy as np
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam


class Autoencoder(Model):
    def _init_(self, latent_dim, shape):
        super(Autoencoder, self)._init()  # Call super().init_()
        self.latent_dim = latent_dim
        self.shape = shape
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=shape),
            layers.Conv2D(128, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(128, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.1),
            layers.Conv2D(64, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(64, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.1),
            layers.Conv2D(32, (2, 2), activation='relu', padding='same'),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),
            layers.UpSampling2D((2, 2))  # Adjusted to match the input shape
        ])
        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(32, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.UpSampling2D((2, 2)),  # Adjusted to match the downsampling factor
            layers.Dropout(0.1),
            layers.Conv2DTranspose(64, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2DTranspose(64, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.UpSampling2D((2, 2)),  # Adjusted to match the downsampling factor
            layers.Dropout(0.1),
            layers.Conv2DTranspose(128, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2DTranspose(128, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Dropout(0.1),
            layers.Conv2DTranspose(shape[-1], (3, 3), activation='sigmoid', padding='same')
        ])
        self.build((None,) + shape)  # Explicitly build the model

    def call(self, x):
        encoded = tf.cast(x, tf.float32)  # Cast input data to float32
        encoded = self.encoder(encoded)
        decoded = self.decoder(encoded)
        return decoded

# Assuming shape is properly defined as (80, 80, 3)
shape = (80, 80, 3)
latent_dim = 4
autoencoder = Autoencoder(latent_dim, shape)

# Compile the model (add loss function and optimizer)
autoencoder.compile(loss='mse',
                          optimizer=Adam(learning_rate=0.1),
                          metrics=['accuracy'])

# Assuming X_train and X_test are properly defined
history=autoencoder.fit(X_train, X_train,
                epochs=50,
                shuffle=True,
                validation_data=(X_test, X_test)
                )

Total number of images: 8610
Number of training images: 7749
Number of testing images: 861


100%|██████████| 1937/1937 [00:00<00:00, 11206.78it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (7749, 2) + inhomogeneous part.